In [ ]:
import sys
import os
import desc_bpz
cwd, _ = os.path.split("__file__")
parent_dir = os.path.abspath(os.path.join(cwd, ".."))
print(parent_dir)
os.environ["BPZDATAPATH"] = os.path.join(parent_dir, "DESC_BPZ", "src", "desc_bpz", "data_files")


## BPZ Tests

In [ ]:
cwd = os.path.abspath(os.path.split("__file__")[0])
parent_dir = os.path.join(cwd, "..")
# os.environ["LEPHAREDIR"] = os.path.join(parent_dir, "LEPHARE")
# os.environ["LEPHAREWORK"] = os.path.join(cwd)
os.environ["BPZDATAPATH"] = os.path.abspath(os.path.join(parent_dir, "DESC_BPZ", "src", "desc_bpz", "data_files"))
data_dir = os.environ.get("BPZDATAPATH", None)
fil_dir = None if data_dir is None else os.path.join(data_dir, "FILTER")
ab_dir = None if data_dir is None else os.path.join(data_dir, "AB")
sed_dir = None if data_dir is None else os.path.join(data_dir, "SED")
paths = [cwd, parent_dir, os.environ.get("BPZDATAPATH", None)]
for path in paths:
    print(path)

In [ ]:
import numpy as np
from desc_bpz.bpz_tools_py3 import prior_with_dict, prior

def test_old_vs_new_prior():
    zgrid = np.arange(0, 10.01, 0.1)
    mags = np.array([20.5, 21., 22.])
    # copy params needed for hdfn_gen prior
    paramdict = dict(fo_arr=[0.35, 0.5], kt_arr=[0.45, 0.147],
                     alpha_arr=[2.465, 1.806, 0.906],
                     zo_arr=[0.431, 0.390, 0.0626],
                     km_arr=[0.0913, 0.0636, 0.123],
                     m0=20.0,
                     numpertype=[1, 2, 3])

    for mag in mags:
        oldprior = prior(zgrid, mag, 'hdfn_gen')
        newprior = prior_with_dict(zgrid, mag, paramdict)
        assert (oldprior == newprior).all()
test_old_vs_new_prior()

In [ ]:
import os
import pytest
# dirname, _ = os.path.split(__file__)
# parent = os.path.join(dirname, '../src/desc_bpz/data_files/')


# This test has to go first 
def test_no_path():
    tmp = os.environ.get("BPZDATAPATH")
    if tmp is not None:
        del os.environ["BPZDATAPATH"]

    import desc_bpz.paths

    # Future proof in case a later test import the
    # module and are called first
    if desc_bpz.paths.data_dir is not None:
        # In this case first restore the env var
        if tmp is not None:
            os.environ["BPZDATAPATH"] = tmp
        # then mark the test as skipped
        pytest.skip("desc_bpz already imported by another test and path set")

    # should fail if BPZDATAPATH not set and we have not set manually
    with pytest.raises(RuntimeError):
        print(desc_bpz.paths.get_ab_file("El_B2004a.DC2LSST_g.AB"))

    # restore env var
    if tmp is not None:
        os.environ["BPZDATAPATH"] = tmp


def test_path():
    import desc_bpz.paths
    data_dir = os.environ.get("BPZDATAPATH", None)
    fil_dir = None if data_dir is None else data_dir + "/FILTER/"
    ab_dir = None if data_dir is None else data_dir + "/AB/"
    sed_dir = None if data_dir is None else data_dir + "/SED/"
    print(os.environ.get("BPZDATAPATH"))
    assert os.path.exists(desc_bpz.paths.get_ab_file("El_B2004a.DC2LSST_g.AB"))
    assert os.path.exists(desc_bpz.paths.get_fil_file("DC2LSST_g.res"))
    assert os.path.exists(desc_bpz.paths.get_sed_file("CWWSB4.list"))


test_no_path()
test_path()


In [ ]:
import os
import git

base_dir = os.path.abspath(git.Repo('.', search_parent_directories=True).working_tree_dir)
os.environ["BPZDATAPATH"] = os.path.abspath(os.path.join(base_dir, "DESC_BPZ", "src", "desc_bpz", "data_files"))
path = "$BPZDATAPATH"
print(os.path.expandvars(path))

## Roman Filter Transmission Data

In [6]:
import pandas as pd
import numpy as np
aperature = np.pi * 1.2**2
df = pd.read_csv("Roman_effarea_20210614.csv").drop(columns=['SNPrism', 'Grism_1stOrder', 'Grism_0thOrder'])
filters = ['F062', 'F087', 'F106', 'F129', 'F158', 'F184', 'F146', 'F213']
df["Wave"] *= 10000 # convert from micron to angstrom units for BPZ
df["Wave"] = df["Wave"].astype(int) # convert from micron to angstrom units
print("Gains above one:")
for filter in filters:
    df[filter] = df[filter] / aperature # effective area to gain (1e-10 is needed to convert from angstrom to meters)
    print(df[filter][df[filter]>1])

Gains above one:
Series([], Name: F062, dtype: float64)
Series([], Name: F087, dtype: float64)
Series([], Name: F106, dtype: float64)
Series([], Name: F129, dtype: float64)
Series([], Name: F158, dtype: float64)
Series([], Name: F184, dtype: float64)
Series([], Name: F146, dtype: float64)
Series([], Name: F213, dtype: float64)


In [7]:

for filter in filters:
    df[["Wave", filter]].rename(columns={"Wave": "#Wavelength(AA)", filter: "Transmission"}).to_csv(f"data_files/RST_{filter}.res", sep=" ", index=False)

In [16]:
from astropy import units as u
from astropy.constants import c

In [17]:
equivalencies = [ (u.Jy, u.erg/u.s/u.cm**2/u.AA, lambda x: 1e-33 * x / c.value , lambda x: c.value * x / 1e-33) ] 
LSST_units = 1e-9*u.Jy
flux_conversion = (LSST_units).to(u.erg/u.s/u.cm**2/u.AA, equivalencies=equivalencies)
print(type(flux_conversion))

<class 'astropy.units.quantity.Quantity'>
